#Part 1: Prerequisites and Initial Setup

In [ ]:
# STEP 1: Install Google Generative AI base library
%pip install -q google-generativeai
print("✅ google-generativeai installed.")

# STEP 2: Install Google Agent Development Kit (ADK)
%pip install -q google-adk
print("✅ google-adk installed.")

# STEP 2.1: Install LangSmith for Observability (Optional, but Recommended)
%pip install -q langsmith
print("✅ langsmith installed.")

#Part 2: Full System Script (Skeleton and Key Areas for Your Case)

In [ ]:
# STEP 3: Necessary Imports
import os
from google.colab import userdata
from google import genai as google_genai_core
from datetime import date, datetime, timedelta # Added datetime, timedelta
import textwrap
from IPython.display import display, Markdown
import warnings
import json # For structured inputs/outputs if needed

# ADK Imports
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search # May or may not be used by all new agents
from google.genai import types as google_genai_types

# LangSmith Imports (for Observability)
from langsmith import Client as LangSmithClient # Renamed to avoid conflict if 'Client' is used elsewhere
from langsmith.run_trees import RunTree
# from langsmith.wrappers import wrap_openai # Not directly used here, but good to know
# from langsmith.utils import traceable # Decorator for easier tracing

print("✅ Libraries imported.")

### STEP 4: Configure Google Gemini API Key & LangSmith (if using)

In [ ]:
# --- Google Gemini API Key Configuration ---
GEMINI_API_KEY_SECRET_NAME = 'GOOGLE_API_KEY_SECONDARY' # BASADO EN TU SCREENSHOT, ESTE SECRETO ESTÁ ACTIVADO
try:
    GOOGLE_API_KEY_VALUE = userdata.get(GEMINI_API_KEY_SECRET_NAME)
    if not GOOGLE_API_KEY_VALUE:
        raise ValueError(f"Secret '{GEMINI_API_KEY_SECRET_NAME}' not found in Colab secrets or is empty.")

    # Establecer la variable de entorno es la forma principal para que ADK y genai la encuentren.
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY_VALUE

    # No es necesaria la llamada a google_genai_core.configure() si la variable de entorno GOOGLE_API_KEY está seteada.
    # Las bibliotecas de Google la recogerán automáticamente.
    print(f"✅ Google Gemini API Key from secret '{GEMINI_API_KEY_SECRET_NAME}' set in environment.")

except Exception as e:
    print(f"❌ Error configuring Google Gemini API Key: {e}. Ensure '{GEMINI_API_KEY_SECRET_NAME}' is set, active, and has a valid key in Colab secrets.")


In [ ]:
# --- LangSmith Configuration (Optional, but Recommended) ---
LANGSMITH_API_KEY_SECRET_NAME = 'LANGSMITH_API_KEY_FAMILY_PROJECT' # BASADO EN TU SCREENSHOT, ESTE SECRETO ESTÁ ACTIVADO
LANGSMITH_PROJECT_NAME = "ADK_Family_Routine_Optimizer_V1" # O el nombre de proyecto que prefieras en LangSmith

try:
    LANGSMITH_API_KEY_VALUE = userdata.get(LANGSMITH_API_KEY_SECRET_NAME)
    if not LANGSMITH_API_KEY_VALUE:
        raise ValueError(f"Secret '{LANGSMITH_API_KEY_SECRET_NAME}' for LangSmith not found in Colab secrets or is empty.")

    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY_VALUE
    os.environ["LANGCHAIN_PROJECT"] = LANGSMITH_PROJECT_NAME

    # La inicialización explícita de LangSmithClient() no es estrictamente necesaria
    # si las variables de entorno están bien configuradas y solo usas RunTree.
    # Si empiezas a usar otras funcionalidades de `langsmith` podrías necesitarla.
    # from langsmith import Client as LangSmithClient
    # langsmith_client = LangSmithClient()

    print(f"✅ LangSmith configured for project: {os.environ['LANGCHAIN_PROJECT']}.")
    print(f"   LANGCHAIN_TRACING_V2: {os.environ.get('LANGCHAIN_TRACING_V2')}")
    print(f"   LANGCHAIN_API_KEY: {'*' * (len(os.environ.get('LANGCHAIN_API_KEY', '')) - 4) + os.environ.get('LANGCHAIN_API_KEY', '')[-4:] if os.environ.get('LANGCHAIN_API_KEY') else 'Not Set'}")


except Exception as e:
    print(f"ℹ️ LangSmith not configured or error during setup: {e}. Tracing will be disabled.")
    os.environ["LANGCHAIN_TRACING_V2"] = "false"


In [ ]:
# STEP 5: Configure Warnings (Optional)
warnings.filterwarnings("ignore")
print("✅ Warnings configured to be ignored.")

### STEP 6: Helper Functions (call_agent, to_markdown)
--- call_agent function (instrumented for LangSmith) ---
*   HACK: This function is crucial. Ensure session handling is correct for ADK.
*   The version from our previous discussion with session creation should be used.

In [ ]:
# PASO 6: Helper Functions (call_agent ya está corregido para AttributeError)

def call_agent(agent_instance: Agent, message_text: str, target_language: str = "en") -> str:
    """
    Sends a message to an ADK agent via a Runner, returns the final response,
    and traces the execution with LangSmith.
    Includes target_language in inputs for LangSmith.
    """
    langsmith_tracing_active = os.environ.get("LANGCHAIN_TRACING_V2", "false") == "true"
    rt: RunTree = None # Type hint para claridad

    if langsmith_tracing_active:
        try:
            rt = RunTree(
                name=f"Agent_ADK_{agent_instance.name}",
                run_type="chain",
                inputs={
                    "agent_name": agent_instance.name,
                    "model_used": agent_instance.model,
                    "initial_message": message_text,
                    "target_language_for_output": target_language,
                    "agent_instruction_preview": agent_instance.instruction[:300] + "..." if agent_instance.instruction else "N/A"
                },
                project_name=os.environ.get("LANGCHAIN_PROJECT", "Default_ADK_Family_Routine"),
            )
        except Exception as e:
            print(f"⚠️ LangSmith RunTree creation failed: {e}")
            langsmith_tracing_active = False

    session_service = InMemorySessionService()
    current_user_id = "family_user_01"
    current_session_id = f"session_{agent_instance.name.replace('_', '')}_{datetime.now().strftime('%Y%m%d%H%M%S%f')}"

    try:
        session_service.create_session(
            app_name=agent_instance.name,
            user_id=current_user_id,
            session_id=current_session_id
        )
    except Exception as e:
        error_message = f"Session config error for {agent_instance.name}: {e}"
        print(f"⚠️ {error_message}")
        if langsmith_tracing_active and rt:
            rt.end(error=error_message)
        return error_message

    runner_instance = Runner(
        agent=agent_instance,
        app_name=agent_instance.name,
        session_service=session_service
    )
    input_content = google_genai_types.Content(
        role="user",
        parts=[google_genai_types.Part(text=message_text)]
    )
    final_response_text = ""

    try:
        for event in runner_instance.run(user_id=current_user_id, session_id=current_session_id, new_message=input_content):
            if event.is_final_response():
                if event.content and event.content.parts:
                    for part_event in event.content.parts:
                        if part_event.text:
                            final_response_text += part_event.text
                final_response_text += "\n"
        final_response_text = final_response_text.strip()
        if langsmith_tracing_active and rt:
            rt.end(outputs={"response": final_response_text})
    except Exception as e:
        error_message = f"Execution error in {agent_instance.name}: {e}"
        print(f"❌ {error_message}")
        if langsmith_tracing_active and rt:
            rt.end(error=str(e))
        final_response_text = error_message

    return final_response_text

def to_markdown(text: str):
    if not isinstance(text, str):
        text = str(text)
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

print("✅ Helper functions defined (call_agent unchanged from previous fix).")

### STEP 7: Agent Definitions for Family Routine Automation
*   HACK: Instructions should be in ENGLISH.
*   The agent will be instructed to PROVIDE OUTPUT in pt-BR or es-LA.

In [ ]:
# ===============================
# AGENT 1: ADAPTATIVE ROUTINE PLANNER (AgenteRoutina)
# ===============================
# CORREGIDO: Añadidos mother_name y father_name como parámetros
def define_routine_planner_agent(current_date_str: str,
                                 child_name: str = "Zoe",
                                 child_age_str: str = "2 years 8 months",
                                 mother_name: str = "Mom", # NUEVO PARÁMETRO
                                 father_name: str = "Dad", # NUEVO PARÁMETRO
                                 previous_day_feedback: str = "No specific feedback.",
                                 current_wakeup_time: str = "N/A",
                                 target_language: str = "pt-BR") -> Agent:
    # HACK: Dynamically build a rich instruction prompt
    instruction = f"""You are 'ZenRoutineMaster', an AI assistant for {child_name}, a {child_age_str} girl.
Today is {current_date_str}.
Her mother's name is {mother_name}. Her father's name is {father_name}.

**Previous Day's Feedback & Observations (if any):**
{previous_day_feedback}

**Today's Actual Wake-up Time:** {current_wakeup_time}

**Your Task:**
Create an adaptive daily schedule for {child_name} for TODAY.
The schedule should be flexible, with suggested 30-60 minute time blocks.
Output the schedule in {target_language}.

**Key Considerations:**
1.  **Adaptability:** Adjust based on `current_wakeup_time` and `previous_day_feedback`.
    *   If wake-up is significantly later than usual, gently shift subsequent activities.
    *   Implement gradual compensation if sleep patterns are off (e.g., 15-min adjustments per day to guide back to a target).
2.  **Child's Needs:**
    *   {child_name} is {child_age_str} and requires consistent (but flexible) meal times, nap times, and activity periods.
    *   Breastfeeding is on demand and a key part of her routine and emotional bonding; integrate it naturally around meals and before sleep.
3.  **Structure:** Suggest blocks for:
    *   Wake-up & Connection Time (incl. breastfeeding if applicable)
    *   Breakfast
    *   Morning Activity (educational/developmental)
    *   Free Play / Independent Play
    *   Lunch
    *   Nap Time (estimate duration based on age, previous day, and wake-up time)
    *   Quiet Post-Nap Activity
    *   Snack Time
    *   Afternoon Activity / Outing (e.g., walk with parents)
    *   Dinner
    *   Evening Wind-Down Routine & Bedtime (estimate target)
4.  **Output Format:**
    *   A clear, itemized list of time blocks and suggested activities in {target_language}.
    *   Use gentle, positive language with appropriate emojis.
    *   If you suggest a "Type A (High Energy)" or "Type B (Low Energy)" day based on feedback, state it.

**Example Output Snippet (for pt-BR):**
Manhã (09:00 - 09:45): Atividade de desenvolvimento motor fino 🧩 - Construir com blocos.
Almoço (12:00 - 12:45): Refeição nutritiva e tranquila. 🥦🥕
"""
    return Agent(
        name="AdaptiveRoutinePlanner",
        model="gemini-1.5-flash-latest",
        description=f"Generates an adaptive daily routine for a {child_age_str} child.",
        instruction=instruction
    )

In [ ]:
# ===============================
# AGENT 2: NUTRITIONAL CHEFF ASSISTANT (AgenteAlimentacion)
# ===============================
# CORREGIDO: Eliminado el comentario inválido dentro de la f-string
def define_meal_planner_agent(available_ingredients: list,
                              family_preferences: str,
                              child_dietary_needs: str,
                              child_name: str,
                              target_language: str = "pt-BR",
                              planning_mode: str = "daily") -> Agent:
    instruction = f"""You are 'NutriChefAI', a nutritional assistant for a family with a toddler ({child_dietary_needs}), whose name is {child_name}.
Your goal is to create healthy and easy-to-prepare meal plans.
Output the meal plan in {target_language}.

**Current Food Inventory:**
{', '.join(available_ingredients) if available_ingredients else 'Assume a moderately stocked pantry and fridge.'}

**Family Preferences:**
{family_preferences}

**Current Planning Mode:** {planning_mode}

**Your Task based on Planning Mode:**
*   **If Mode is 'daily':**
    1.  Generate a balanced meal plan for TODAY for breakfast, lunch, snack, and dinner for the whole family (parents and {child_name}).
    2.  Provide specific, simple recipe ideas for {child_name}, considering {child_dietary_needs}.
    3.  Suggest if a family meal can be adapted for {child_name}.
    4.  Prioritize using `available_ingredients`.
    5.  Minimize preparation time.
*   **If Mode is 'emergency_3_ingredient':**
    1.  The user will provide 2-3 main ingredients.
    2.  Suggest 1-2 ultra-simple and quick meal ideas using ONLY those ingredients (plus common staples like salt, oil) appropriate for {child_name} if possible.
*   **If Mode is 'weekly_batch_cooking_prep':**
    1.  Suggest 2-3 main dishes that can be cooked in larger batches over the weekend to cover several meals during the week for the parents.
    2.  Suggest 1-2 components that can be prepped for {child_name}'s meals (e.g., steamed vegetables, cooked grains).

**Output Format (for 'daily' mode in {target_language}):**
**Café da Manhã (Exemplo pt-BR):**
*   Pais: Omelete com espinafre e torradas.
*   {child_name}: Papa de aveia com frutas amassadas (banana, mamão). (Considerar lactancia)
**Almoço:**
*   Todos: Frango desfiado com purê de batata doce e brócolis cozidos. (Adaptar textura para {child_name})
**Lanche da Tarde (Exemplo pt-BR):**
*   {child_name}: Palitos de maçã com pasta de amendoim (se não houver alergias e for apropriado para a idade).
*   Pais: Frutas ou iogurte.
**Jantar (Exemplo pt-BR):**
*   Todos: Sopa de legumes nutritiva com pedaços de pão integral. (Garantir que os legumes estejam bem cozidos para {child_name}).
"""
    return Agent(
        name="NutritionalChefAssistant",
        model="gemini-1.5-flash-latest",
        description="Generates family meal plans considering toddler needs and available ingredients.",
        instruction=instruction
    )


In [ ]:
# ===============================
# AGENT 3: AUTONOMY DEVELOPMENT COORDINATOR (AgenteActividades)
# ===============================
# (Revisar y asegurarse de que no hay comentarios inválidos en su f-string)
def define_activity_coordinator_agent(available_home_resources: list,
                                      current_weather_brief: str,
                                      caregiver_availability: str,
                                      child_name: str,
                                      child_age_str: str,
                                      child_mood: str = "neutral",
                                      target_language: str = "pt-BR") -> Agent:
    instruction = f"""You are 'PlaySpark', an AI activity coordinator for {child_name}, a {child_age_str} girl.
Your goal is to suggest 3-5 engaging, screen-free activities that promote development and autonomy.
Output suggestions in {target_language}.

**Available Home Resources/Toys (example):**
{', '.join(available_home_resources) if available_home_resources else 'Blocks, crayons, paper, stuffed animals, balls, household items like pots and spoons.'}

**Current Weather (brief):** {current_weather_brief}
**Caregiver's Availability for Engagement:** {caregiver_availability}
**Child's Current Mood (if known):** {child_mood}

**Your Task:**
1.  Suggest 3-5 varied activities for TODAY for {child_name}.
2.  Activities MUST be screen-free.
3.  Primarily use `available_home_resources`.
4.  Consider `current_weather_brief`.
5.  Adapt to `caregiver_availability`.
6.  If `child_mood` suggests low energy, propose calmer activities for {child_name}.
7.  Include at least one activity that fosters {child_name}'s autonomy (e.g., simple self-care task, choosing between two offered toys).
8.  Optionally, suggest an "imitation activity" where {child_name} can safely mirror a parent's task.
9.  Briefly explain the developmental benefit of one of the suggested activities for {child_name}.

**Output Format (in {target_language}):**
**Sugestões de Atividades para Hoje (Exemplo pt-BR):**
1.  **Blocos Criativos (Jogo Autônomo):** Deixe {child_name} explorar os blocos de montar livremente. *Benefício: Desenvolve a coordenação motora fina e a criatividade.*
2.  **Caça ao Tesouro Sensorial (Com Cuidador):** Esconda objetos com texturas diferentes pela sala para {child_name} encontrar.
3.  **Ajudante na Cozinha (Imitação):** Se estiver preparando algo simples, deixe {child_name} "ajudar" com utensílios de brinquedo ao seu lado.
4.  **Leitura Interativa (Com Cuidador):** Escolham um livro e leiam juntos, apontando para as figuras e fazendo os sons dos animais.
5.  **Torre de Almofadas (Jogo Autônomo/Com Cuidador):** Empilhar almofadas e derrubá-las. Diversão garantida!
"""
    return Agent(
        name="AutonomyDevelopmentCoordinator",
        model="gemini-1.5-flash-latest",
        description="Suggests screen-free developmental activities for a toddler.",
        instruction=instruction
    )


In [ ]:
# ===============================
# AGENT 4: PROGRESS COACH & EVALUATOR (AgenteCoach)
# ===============================
# (Revisar y asegurarse de que no hay comentarios inválidos en su f-string)
def define_progress_coach_agent(weekly_routine_logs: str,
                                caregiver_feedback_summary: str,
                                family_goals: str,
                                child_name: str,
                                target_language: str = "pt-BR") -> Agent:
    modelo_coach = "gemini-1.5-pro-latest"
    instruction = f"""You are 'GrowthGuideAI', a family coach specializing in child development and routine optimization.
You are reviewing the past week for {child_name}'s family.
Output your report and recommendations in {target_language}.

**Weekly Routine Logs (summary or key data points):**
{weekly_routine_logs}

**Caregiver's Feedback Summary for the Week:**
{caregiver_feedback_summary}

**Stated Family Goals:**
{family_goals}

**Your Task:**
1.  Analyze the provided `weekly_routine_logs` and `caregiver_feedback_summary`.
2.  Assess progress towards `family_goals`.
3.  Identify 1-2 patterns or areas of success from the past week concerning {child_name}.
4.  Identify 1-2 areas desafíos (challenges) or points of potential conflict/overload.
5.  Provide 2-3 actionable, positive, and empathetic recommendations for a_ajustes to the routine, activities, or meal plans for the UPCOMING week for {child_name}.
6.  If applicable, suggest a "baby step" towards future delegation related to {child_name}'s care.
7.  Maintain a supportive and encouraging tone.

**Output Format (in {target_language}):**
**Relatório Semanal e Recomendações (Exemplo pt-BR):**
**Pontos Positivos da Semana:**
*   Parabéns! Parece que {child_name} se adaptou bem à nova atividade de pintura com os dedos.
*   A consistência nos horários das refeições principais foi um sucesso.
**Pequenos Desafios e Observações:**
*   As noites de terça e quinta foram um pouco mais agitadas para {child_name}, conforme o feedback.
*   Pode ser útil variar um pouco mais os lanches da tarde.
**Recomendações para a Próxima Semana:**
1.  **Rotina Noturna:** Considerem adicionar um banho morno ao ritual noturno nas noites de terça e quinta para ajudar {child_name} a relaxar.
2.  **Lanches:** O AgenteAlimentacion pode ser instruído a gerar 3 opções de lanches variados para a semana, facilitando a escolha.
3.  **Marco de Desenvolvimento:** Continuem incentivando {child_name} a guardar um brinquedo após o uso, é um ótimo passo para a autonomia!
"""
    return Agent(
        name="ProgressCoachEvaluator",
        model=modelo_coach,
        description="Analyzes weekly progress, provides feedback and recommendations for family routine.",
        instruction=instruction
    )

print("✅ Agent definitions updated (SyntaxError in f-string comments fixed).")

In [ ]:
# ===============================
# POTENTIAL AGENT 5: ORCHESTRATOR Agent....
# ===============================

# HACK: The orchestrator would be an Agent whose "tools" are the other agents.
# Its instruction would be about managing the overall daily/weekly flow.
# For this iteration, we will do manual orchestration in STEP 8.
# def define_orchestrator_agent(...) -> Agent: ...

print("✅ Agent definitions loaded (skeletons for your case).")


# STEP 8: Main Orchestration Logic (Adapted for Family Routine)

In [ ]:
# PASO 8: Main Orchestration Logic (Adapted for Family Routine - MODIFIED)

print(f"\n🚀 Initializing Family Routine Automation System 🧸")
current_date_str = date.today().strftime("%A, %d %B %Y")
print(f"Today is: {current_date_str}")

user_selected_language = input("Select output language (pt-BR / es-LA): ").strip()
if user_selected_language not in ["pt-BR", "es-LA"]:
    user_selected_language = "pt-BR"
print(f"Language for outputs: {user_selected_language}")

# --- User Inputs & Configuration ---
child_name_config = "Zoe" # Configurable child's name
child_age_config = "2 years and 8 months" # Configurable child's age
mother_name_config = input(f"Enter Mother's name (e.g., Maria): ") or "Maria" # Get mother's name
father_name_config = input(f"Enter Father's name (e.g., João): ") or "João" # Get father's name

simulated_previous_day_feedback = "Zoe slept well but woke up a bit earlier than usual. Morning mood was great. Afternoon nap was a bit short."
simulated_wakeup_time_today = input(f"What time did {child_name_config} wake up today? (e.g., 7:00 AM): ") or "7:15 AM"

simulated_ingredients = ["chicken breast", "broccoli", "sweet potato", "oats", "banana", "apple", "pasta", "eggs", "spinach"]
simulated_family_preferences = "We like quick meals. Mom is vegetarian some days. Dad eats everything. Zoe is not keen on very spicy food."
simulated_child_dietary_needs = "Toddler balanced diet, needs soft textures for some foods, still breastfeeding on demand."

simulated_home_resources = ["building blocks", "crayons and paper", "soft ball", "story books", "puzzles (large pieces)"]
simulated_weather = "Sunny with a light breeze"
simulated_caregiver_availability = "Mom has focused work blocks from 10 AM - 12 PM and 2 PM - 4 PM. Dad can help with short activities."

print(f"\n--- Running Daily Routine Generation for {child_name_config} ---")

# 1. AgenteRutina
print("\n🔄 Stage 1: Generating Adaptive Routine with RoutinePlanner Agent...")
routine_planner_agent_instance = define_routine_planner_agent(
    current_date_str=current_date_str,
    child_name=child_name_config,             # Usar variable de configuración
    child_age_str=child_age_config,          # Usar variable de configuración
    mother_name=mother_name_config,          # PASAR EL VALOR
    father_name=father_name_config,          # PASAR EL VALOR
    previous_day_feedback=simulated_previous_day_feedback,
    current_wakeup_time=simulated_wakeup_time_today,
    target_language=user_selected_language
)
routine_message = f"Please generate today's adaptive routine for {child_name_config}."
daily_schedule_output = call_agent(routine_planner_agent_instance, routine_message, target_language=user_selected_language)
print(f"\n--- 📅 Output from Agent 1 (RoutinePlanner) in {user_selected_language} ---")
display(to_markdown(daily_schedule_output))
print("----------------------------------------------------\n")

# 2. AgenteAlimentacion
print("🔄 Stage 2: Planning Meals with NutritionalChef Agent...")
meal_planner_agent_instance = define_meal_planner_agent(
    available_ingredients=simulated_ingredients,
    family_preferences=simulated_family_preferences,
    child_dietary_needs=simulated_child_dietary_needs,
    child_name=child_name_config, # Pasar el nombre del niño
    target_language=user_selected_language,
    planning_mode="daily"
)
meal_plan_message = "Generate the daily meal plan based on the configured inputs."
daily_meal_plan_output = call_agent(meal_planner_agent_instance, meal_plan_message, target_language=user_selected_language)
print(f"\n--- 🍲 Output from Agent 2 (NutritionalChef) in {user_selected_language} ---")
display(to_markdown(daily_meal_plan_output))
print("----------------------------------------------------\n")

# 3. AgenteActividades
print("🔄 Stage 3: Suggesting Activities with ActivityCoordinator Agent...")
activity_agent_instance = define_activity_coordinator_agent(
    available_home_resources=simulated_home_resources,
    current_weather_brief=simulated_weather,
    caregiver_availability=simulated_caregiver_availability,
    child_name=child_name_config, # Pasar el nombre del niño
    child_age_str=child_age_config, # Pasar la edad del niño
    child_mood="happy",
    target_language=user_selected_language
)
activity_message = "Suggest suitable activities for today."
daily_activities_output = call_agent(activity_agent_instance, activity_message, target_language=user_selected_language)
print(f"\n--- 🎨 Output from Agent 3 (ActivityCoordinator) in {user_selected_language} ---")
display(to_markdown(daily_activities_output))
print("----------------------------------------------------\n")

# 4. AgenteCoach
run_coach_agent_today = False
if run_coach_agent_today:
    print("🔄 Stage 4: Generating Weekly Insights with ProgressCoach Agent (Demo Run)...")
    dummy_weekly_logs = f"Week of {date.today() - timedelta(days=7)} to {date.today()}:\n- {child_name_config}'s average wake-up: 7:30 AM\n- Naps: Generally 1.5 hours, some shorter.\n- Meals: Good intake, new food 'broccoli' tried.\n- Activities: Loved outdoor walks, less interest in puzzles this week."
    dummy_caregiver_feedback = "Overall a good week. Managing work and Zoe's needs is still a juggle. Transitions to nap time were a bit challenging on Wed/Thu. Felt more organized with meal ideas."
    dummy_family_goals = f"Maintain consistent nap schedule for {child_name_config}, introduce 1 new vegetable, ensure 30 mins outdoor time daily, reduce mother's operational load for routine planning."

    coach_agent_instance = define_progress_coach_agent(
        weekly_routine_logs=dummy_weekly_logs,
        caregiver_feedback_summary=dummy_caregiver_feedback,
        family_goals=dummy_family_goals,
        child_name=child_name_config, # Pasar el nombre del niño
        target_language=user_selected_language
    )
    coach_message = "Provide weekly report and recommendations."
    weekly_coach_report_output = call_agent(coach_agent_instance, coach_message, target_language=user_selected_language)
    print(f"\n--- 📊 Output from Agent 4 (ProgressCoach) in {user_selected_language} ---")
    display(to_markdown(weekly_coach_report_output))
    print("----------------------------------------------------\n")

print("🎉 Family Routine Automation System - Daily Run Complete! 🎉")